In [70]:
import pandas as pd
import os

# cwd = '/Users/stjames/Dropbox/Pablo/data/eoir_dataset/'

# os.chdir(cwd)

df = pd.read_csv('/Volumes/TOSHIBA/foia_trac_reports_feb_23/A_TblCaseIdentifier.csv', sep='\t',on_bad_lines='skip')
df.shape

(1318929, 3)

In [71]:
df.head()

,IDNCASEID,IDNCASE,CASE_ID
0,1,2046920.0,SR
1,2,2046921.0,SR
2,3,2046922.0,SR
3,4,2046924.0,SR
4,5,2046925.0,SR


In [72]:
from sqlalchemy import create_engine


engine = create_engine('postgresql://localhost:5432/postgres', isolation_level="AUTOCOMMIT")
with engine.connect() as conn:
    try:
        conn.execute("""
        SELECT pg_terminate_backend(pg_stat_activity.pid)
            FROM pg_stat_activity
            WHERE pg_stat_activity.datname = 'eoir' AND pid <> pg_backend_pid();
        """)
    except:
        pass
    conn.execute("""
        DROP DATABASE IF EXISTS eoir;
    """)
    conn.execute("""
        CREATE DATABASE eoir;
    """)



In [73]:
df = df.dropna()

In [74]:
df.columns

Index(['IDNCASEID', 'IDNCASE', 'CASE_ID'], dtype='object')

In [75]:
engine = create_engine('postgresql://localhost:5432/eoir', isolation_level="AUTOCOMMIT")
with engine.connect() as conn:
    conn.execute("""
        CREATE TABLE public.case_identifier (
            id serial NOT NULL,
            IDNCASEID int,
            IDNCASE int,
            CASE_ID text,
            PRIMARY KEY ("id")
        );
    """)

In [76]:
import sqlalchemy

engine = create_engine('postgresql://localhost:5432/eoir', isolation_level="AUTOCOMMIT")

df.to_sql('case_identifier',
          engine,
          if_exists='replace',
          index_label='id',
          chunksize=10000,
          dtype= {
            'IDNCASEID': sqlalchemy.types.INTEGER(), 
            'IDNCASE':  sqlalchemy.types.INTEGER(),
            'CASE_ID': sqlalchemy.types.TEXT(),
          })

# Add indices to speed up processing
with engine.connect() as conn:
    conn.execute("""
        CREATE INDEX idx_idncase_id ON case_identifier ("IDNCASE");
    """)

In [77]:
import sqlalchemy


# Create a connection to the PostgreSQL database
engine = sqlalchemy.create_engine('postgresql://localhost:5432/eoir')

# Query the table

query = 'SELECT * FROM case_identifier'

sdf = pd.read_sql(query, engine)

connection.close()


In [78]:
sdf

,id,IDNCASEID,IDNCASE,CASE_ID
0,0,1,2046920,SR
1,1,2,2046921,SR
2,2,3,2046922,SR
3,3,4,2046924,SR
4,4,5,2046925,SR
...,...,...,...,...
1312342,1318924,1694337,11620738,DD
1312343,1318925,1694339,11620823,AU
1312344,1318926,1694340,11620865,DD
1312345,1318927,1694341,11620936,AU
